In [1]:
import pandas as pd
import numpy as np
import scipy.sparse as sp
import json

In [2]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [3]:
def read(start, end):
    path = '/content/drive/My Drive/MusicRecommendationSystem/data/10000 Playlists/mpd.slice.' + str(start) + "-" + str(end-1) + '.json'
    d = json.load(open(path, 'r'))
    df = pd.DataFrame.from_dict(d['playlists'], orient='columns')
    return df

In [6]:
f1 = read(0, 1000)
f2 = read(1000, 2000)
f3 = read(2000, 3000)
f4 = read(3000, 4000)
traindata = pd.concat([f1, f2, f3, f4])

print(len(traindata))

4000


In [7]:
traindata.head()

,name,collaborative,pid,modified_at,num_tracks,num_albums,num_followers,tracks,num_edits,duration_ms,num_artists,description
0,Throwbacks,false,0,1493424000,52,47,1,"[{'pos': 0, 'artist_name': 'Missy Elliott', 't...",6,11532414,37,NaN
1,Awesome Playlist,false,1,1506556800,39,23,1,"[{'pos': 0, 'artist_name': 'Survivor', 'track_...",5,11656470,21,NaN
2,korean,false,2,1505692800,64,51,1,"[{'pos': 0, 'artist_name': 'Hoody', 'track_uri...",18,14039958,31,NaN
3,mat,false,3,1501027200,126,107,1,"[{'pos': 0, 'artist_name': 'Camille Saint-Saën...",4,28926058,86,NaN
4,90s,false,4,1401667200,17,16,2,"[{'pos': 0, 'artist_name': 'The Smashing Pumpk...",7,4335282,16,NaN


In [8]:
#Turn user level dataframe into song level dataframe
songUserArray = []
for index, row in traindata.iterrows():
    for track in row['tracks']:
        songUserArray.append([track['track_uri'], track['artist_name'], track['track_name'], row['pid']])
songPlaylist = pd.DataFrame(songUserArray, columns=['trackid', 'artist_name', 'track_name', 'pid'])

print(songPlaylist.shape)
songPlaylist.head(5) 

(266386, 4)


,trackid,artist_name,track_name,pid
0,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,Missy Elliott,Lose Control (feat. Ciara & Fat Man Scoop),0
1,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,Britney Spears,Toxic,0
2,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,Beyoncé,Crazy In Love,0
3,spotify:track:1AWQoqb9bSvzTjaLralEkT,Justin Timberlake,Rock Your Body,0
4,spotify:track:1lzr43nnXAijIGYnCT8M8H,Shaggy,It Wasn't Me,0


In [9]:
songPlaylist['trackindex'] = songPlaylist['trackid'].astype('category').cat.codes
print(len(songPlaylist['trackindex'].unique()))
songPlaylist.head(10)


93139


,trackid,artist_name,track_name,pid,trackindex
0,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,Missy Elliott,Lose Control (feat. Ciara & Fat Man Scoop),0,6012
1,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,Britney Spears,Toxic,0,75211
2,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,Beyoncé,Crazy In Love,0,6463
3,spotify:track:1AWQoqb9bSvzTjaLralEkT,Justin Timberlake,Rock Your Body,0,14099
4,spotify:track:1lzr43nnXAijIGYnCT8M8H,Shaggy,It Wasn't Me,0,21292
5,spotify:track:0XUfyU2QviPAs6bxSpXYG4,Usher,Yeah!,0,6593
6,spotify:track:68vgtRHr7iZHpzGpon6Jlo,Usher,My Boo,0,73389
7,spotify:track:3BxWKCI06eQ5Od8TY2JBeA,The Pussycat Dolls,Buttons,0,38291
8,spotify:track:7H6ev70Weq6DdpZyyTmUXk,Destiny's Child,Say My Name,0,86994
9,spotify:track:2PpruBYCo4H7WOBJ7Q2EwM,OutKast,Hey Ya! - Radio Mix / Club Mix,0,28961


In [10]:
train_withoutfeatures = songPlaylist
train_withoutfeatures.head(10)

,trackid,artist_name,track_name,pid,trackindex
0,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,Missy Elliott,Lose Control (feat. Ciara & Fat Man Scoop),0,6012
1,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,Britney Spears,Toxic,0,75211
2,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,Beyoncé,Crazy In Love,0,6463
3,spotify:track:1AWQoqb9bSvzTjaLralEkT,Justin Timberlake,Rock Your Body,0,14099
4,spotify:track:1lzr43nnXAijIGYnCT8M8H,Shaggy,It Wasn't Me,0,21292
5,spotify:track:0XUfyU2QviPAs6bxSpXYG4,Usher,Yeah!,0,6593
6,spotify:track:68vgtRHr7iZHpzGpon6Jlo,Usher,My Boo,0,73389
7,spotify:track:3BxWKCI06eQ5Od8TY2JBeA,The Pussycat Dolls,Buttons,0,38291
8,spotify:track:7H6ev70Weq6DdpZyyTmUXk,Destiny's Child,Say My Name,0,86994
9,spotify:track:2PpruBYCo4H7WOBJ7Q2EwM,OutKast,Hey Ya! - Radio Mix / Club Mix,0,28961


In [11]:

features = ['Danceability','Instrumentalness','Loudness','Speechiness','Valence','Energy','Mode','Tempo','Livenss','Key','Acousticness']
train = pd.read_csv("/content/drive/My Drive/MusicRecommendationSystem/all_features4k_playlist.csv")
train['trackindex'] = train['trackid'].astype('category').cat.codes
# user_input, item_input, labels = get_features(train,features)
# print(len(user_input),len(item_input))
train.head()

,Unnamed: 0,trackid,artist_name,track_name,duration_ms,album_name,pid,Danceability,Instrumentalness,Loudness,Speechiness,Valence,Energy,Mode,Tempo,Livenss,Key,Acousticness,trackindex
0,0,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,Missy Elliott,Lose Control (feat. Ciara & Fat Man Scoop),226863,The Cookbook,0,0.904,0.006970,-7.105,0.1210,0.810,0.813,0,125.461,0.0471,4,0.03110,6012
1,1,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,Britney Spears,Toxic,198800,In The Zone,0,0.774,0.025000,-3.914,0.1140,0.924,0.838,0,143.040,0.2420,5,0.02490,75211
2,2,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,Beyoncé,Crazy In Love,235933,Dangerously In Love (Alben für die Ewigkeit),0,0.664,0.000000,-6.583,0.2100,0.701,0.758,0,99.259,0.0598,2,0.00238,6463
3,3,spotify:track:1AWQoqb9bSvzTjaLralEkT,Justin Timberlake,Rock Your Body,267266,Justified,0,0.891,0.000234,-6.055,0.1400,0.818,0.714,0,100.972,0.0521,4,0.20200,14099
4,4,spotify:track:1lzr43nnXAijIGYnCT8M8H,Shaggy,It Wasn't Me,227600,Hot Shot,0,0.853,0.000000,-4.596,0.0713,0.654,0.606,1,94.759,0.3130,0,0.05610,21292


In [ ]:
'''
Sparse matrix - Dimensions : (NoOfPlaylists,NoOfSOngs)
Sparse matrix[i][j] = 1 if playlist i contains song j 
                      0 other wise 
                      
    We cannot scale this matrix for large datasets.
'''
# Playlists - Song Matrix 
print(len(traindata), len(train['trackindex'].unique()))
mat = sp.dok_matrix((len(traindata), len(train['trackindex'].unique())), dtype=np.float32)
for pid, trackindex in zip(train['pid'], train['trackindex']):
    try:
      mat[pid, trackindex] = 1.0
    except:
      print(pid)
      print(trackindex)
      break


4000 93139


In [20]:
'''
Song-Feature Matrix:
  Dimensions : (NoOfSongs,NoOfFeatures)
  Later in the Algorithm we initial an songs_embedding layer with with this matrix
'''
#  Song - Feature Matrix : 
print(len(train['trackindex'].unique()), len(features))
song_feature = []
for trackindex in range(0,len(train['trackindex'].unique())):
  rows = train.loc[train['trackindex'] == trackindex]
  x=1
  for index, row in rows.iterrows():
    arr=[]
    for feature in features :
      arr.append(row[feature])
    song_feature.append(arr)
    if x==1: 
      break

print(len(song_feature),len(song_feature[0]))
y=np.array([np.array(xi) for xi in song_feature])
print(y.shape)

93139 11
93139 11
(93139, 11)


In [22]:
'''
In this function , 
      To generate training data we make use of the Dataframe directly passing the intermediate step of creating a sprase Matrix , which ensure "Scalability"
      If the Playlist doesn't contain a song , we can't rule the possibility of song not being a part of Playlist . The creator of the Playlist might not have come-across the song itself. 

      We would like to consider some negative samples  
      Reasons : 
        1. Considering all the negative samples is computationally heavy 
        2. Provide that both positive and neagtive samples to the model to enhance learning. 
''' 

def train_samples(train, num_negatives):
    user_input = []
    item_input = []
    labels = []
    num_playlists = len(traindata)
    print(num_playlists)
    # num_playlists = 1
    num_items = len(train['trackindex'].unique())
    for u in range(0,num_playlists):
      rows = train.loc[train['pid'] == u]
      songs_list = rows["trackindex"].tolist()
      # positive instance
      for i in songs_list:
        user_input.append(u)
        item_input.append(i)
        labels.append(1)
      # negative instances
      for t in range(num_negatives):
          j = np.random.randint(num_items)
          while j in songs_list:
                j = np.random.randint(num_items)
          user_input.append(u)
          item_input.append(j)
          labels.append(0)
    return user_input, item_input, labels


# user_input, item_input, labels = train_samples(train,num_negatives)
# print(user_input[1:10])
# print(item_input[1:10])
# print(labels[1:10])


In [23]:
#now building neural network
import numpy as np
import theano.tensor as T
import keras
from keras import backend as K
from keras import initializers
from keras.models import Sequential, Model, load_model, save_model
from keras.layers.core import Dense, Lambda, Activation
from keras.layers import Embedding, Input, Dense, Reshape, Flatten, Concatenate, concatenate, Multiply
from keras.optimizers import Adam
from keras.regularizers import l2
from time import time
import multiprocessing as mp
import sys
import math

def init_normal(shape, name=None):
    return initializers.normal(shape, scale=0.01, name=name)

def get_model(num_users, num_items, latent_dim):

    user_input = Input(shape=(1,), dtype='int32', name = 'user_input')
    item_input = Input(shape=(1,), dtype='int32', name = 'item_input')

    user_embedding = Embedding(input_dim = num_users, output_dim = latent_dim, name = 'user_embedding', input_length=1)
    item_embedding = Embedding(input_dim = num_items, output_dim = latent_dim, name = 'item_embedding', input_length=6) 
    item_embedding.build(input_shape=(1,)) # the input_shape here has no effect in the build function
    item_embedding.set_weights([y])  
    
    user_latent = Flatten()(user_embedding(user_input))
    item_latent = Flatten()(item_embedding(item_input))
    
    predict_vector = Multiply()([user_latent, item_latent])
    
    prediction = Dense(1, activation='sigmoid', name = 'prediction')(predict_vector)
    
    model = Model([user_input, item_input], prediction)
    return model

# def train_samples(train, num_negatives):
#     user_input = []
#     item_input = []
#     labels = []
#     num_users = train.shape[0]
#     for (u, i) in train.keys():
#         # positive instance
#         user_input.append(u)
#         item_input.append(i)
#         labels.append(1)
#         # negative instances
#         for t in range(num_negatives):
#             j = np.random.randint(num_items)
#             while (u,j) in train.keys():
#                 j = np.random.randint(num_items)
#             user_input.append(u)
#             item_input.append(j)
#             labels.append(0)
#     return user_input, item_input, labels


num_factors = len(features)
num_negatives = 100
learner = 'adam'
learning_rate = 0.001
epochs = 3
batch_size = 200
verbose = 1

    
# Loading data output
num_users, num_items = len(traindata) , len(train['trackindex'].unique()) 
print("Loaded")

# Build model
model = get_model(num_users, num_items, num_factors)
model.compile(optimizer=Adam(lr=learning_rate), loss='binary_crossentropy', metrics=['accuracy'])
print(model.summary())


print("Training")
for epoch in range(epochs):
    user_input, item_input, labels = train_samples(train,num_negatives)
    print(user_input[0],item_input[0]) 
    # Training
    hist = model.fit([np.array(user_input), np.array(item_input)], np.array(labels), validation_split=0.20, batch_size=batch_size, epochs=1, verbose=0, shuffle=True)
    print(hist.history)



Loaded
Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
user_input (InputLayer)         [(None, 1)]          0                                            
__________________________________________________________________________________________________
item_input (InputLayer)         [(None, 1)]          0                                            
__________________________________________________________________________________________________
user_embedding (Embedding)      (None, 1, 11)        44000       user_input[0][0]                 
__________________________________________________________________________________________________
item_embedding (Embedding)      (None, 1, 11)        1024529     item_input[0][0]                 
________________________________________________________________________________

In [24]:
weights = model.get_weights()
print(weights)

[array([[-0.00947232,  0.79815215,  0.27487305, ...,  0.2602465 ,
        -0.0651174 ,  0.74764806],
       [-0.21669593, -0.0540654 , -0.04419631, ...,  0.3151599 ,
        -0.13440458,  0.42042267],
       [ 0.18234152,  0.38664117,  0.27067968, ..., -0.02482796,
        -0.02914313, -0.28906953],
       ...,
       [-0.04859766, -0.02449691,  0.014025  , ..., -0.03608917,
        -0.01798933, -0.03930283],
       [ 0.03038407, -0.04219611, -0.0213288 , ...,  0.04777521,
         0.02990248,  0.03421948],
       [ 0.01647789,  0.04712541,  0.04272785, ..., -0.01310073,
        -0.02733111,  0.01595142]], dtype=float32), array([[  0.41927567,   0.17939901,  -4.2530165 , ...,   0.36705124,
          7.0356684 ,   0.06252268],
       [  0.24517705,   0.12820391,  -8.462692  , ...,   0.2210637 ,
         10.050372  ,   0.13571092],
       [  1.1360853 ,  -0.32195488,  -6.030635  , ...,  -0.14885283,
          6.9335613 ,  -0.0264663 ],
       ...,
       [  0.71312   ,   0.7941698 ,  -5.

In [25]:
import pickle
import os 
os.chdir('/content/drive/My Drive/MusicRecommendationSystem')
with open('weights4kwithfeatures.pickle', 'wb') as f:
    pickle.dump(weights,f)

In [26]:
import os 
import pickle
os.chdir('/content/drive/My Drive/MusicRecommendationSystem')
with open('weights4kwithfeatures.pickle', 'rb') as f:
    model_weights = pickle.load(f)

playlist_matrix = model_weights[0]
print(playlist_matrix.shape)
songs_matrix = model_weights[1]
print(songs_matrix.shape)

(4000, 11)
(93139, 11)


In [27]:
'''
The speciality of these vectors is that the Embedding vectors are learnt from model and the cosine similarity is used to fing similar songs 
'''
def cosine_similarity(word_vectors,track_id,n):
  cosine_dict = {}
  word_list = []
  index = track_id -1 
  target = word_vectors[index,:]
  for i in range(0,word_vectors.shape[0]):
    curr = word_vectors[i]
    cos_sim = np.dot(curr,target)/(np.linalg.norm(curr)*np.linalg.norm(target))
    cosine_dict[i]=cos_sim
  dist_sort=sorted(cosine_dict.items(), key=lambda dist: dist[1],reverse = True)
  for item in dist_sort:
    word_list.append((item[0]+1,item[1]))
  return word_list[0:n]

In [28]:
get = cosine_similarity(songs_matrix,6012,10)
print(get)

[(6012, 1.0), (42358, 0.99999446), (18708, 0.99999315), (36257, 0.99999076), (24395, 0.9999904), (57412, 0.9999902), (48988, 0.99999005), (6592, 0.99998933), (8873, 0.99998826), (55796, 0.999988)]


In [32]:
for track in get : 
  rows = train.loc[train['trackindex'] == track[0]]
  song_name = rows['track_name'].tolist()
  print(track[0],song_name[0])

6012 Lose Control (feat. Ciara & Fat Man Scoop)
42358 Hold Me Up - AM2PM Remix
18708 Free / Into The Mystic - Live From Bonnaroo
36257 Concrete Jungle
24395 Everybody (feat. Jim Jones)
57412 In Check
48988 Stand And Deliver
6592 Breakin' Stuff
8873 Already Home
55796 Mitwa


In [ ]:
'''
Adavantages of this model:
  1.Paramets are trained in neural model so the Performance only gets better with increase in data
  2.Model can be pre-Trained .While making predictions we are making use only the weight matrix 
  3.Ease in adding Optimization layers

Future Work :
  1.Define heuristics to obtain the closes possible Playlist
  2.Apply Optimizations 


Suggestions : 
  Embedding layer -> artits name include all the feaatures 
      representation of artists 
      search based in artists 
      find the balance ; accarcy and speed trade-off 
  giving a paragraph -> convert into representation 
  try differnt things and finf the weights and measure
  diverse input and 
  Tabulate findings 
  Edge-cases 
  include analysis ** with parametrs too 
  Emperical way of measuring the playlist 
  change parameters on model 
  Make sure it's fast enough
  different inputs - artistsnames 

'''

In [ ]:
from keras.layers.core import Dense, Lambda, Activation
from keras.layers import Embedding, Input, Dense, Reshape, Flatten, Concatenate, concatenate, Multiply
import numpy as np 
def Dense_layer(word_vectors,track_id,n,dense_weights):
  cosine_dict = {}
  word_list = []
  index = track_id -1 
  target = word_vectors[index,:]
  for i in range(0,word_vectors.shape[0]):
    curr = word_vectors[i]
    predict_vector = Multiply()([target,curr])
    predict_vector = np.array(predict_vector).reshape((1,len(predict_vector)))
    prediction = np.dot(predict_vector,dense_weights)
    cosine_dict[i]=prediction
  dist_sort=sorted(cosine_dict.items(), key=lambda dist: dist[1],reverse = True)
  for item in dist_sort:
    word_list.append((item[0]+1,item[1]))
  return word_list[0:n]

In [ ]:
dense_weights  = model_weights[2]
get = Dense_layer(songs_matrix,6012,10,dense_weights)
print(get)

[(90662, array([[566.2419]], dtype=float32)), (12191, array([[565.91956]], dtype=float32)), (61273, array([[565.77893]], dtype=float32)), (9356, array([[564.87787]], dtype=float32)), (36091, array([[563.8778]], dtype=float32)), (670, array([[563.7588]], dtype=float32)), (78780, array([[562.72675]], dtype=float32)), (20112, array([[561.359]], dtype=float32)), (90001, array([[374.70786]], dtype=float32)), (37190, array([[372.48254]], dtype=float32))]


In [ ]:
for i in range(0,11):
  print(get[len(get)-1-i])

(52656, array([[-6.7347198]], dtype=float32))
(627, array([[-6.6885653]], dtype=float32))
(53561, array([[-6.6815476]], dtype=float32))
(20284, array([[-6.6702576]], dtype=float32))
(77820, array([[-6.669116]], dtype=float32))
(35584, array([[-6.6628876]], dtype=float32))
(77269, array([[-6.6613035]], dtype=float32))
(47770, array([[-6.654142]], dtype=float32))
(41207, array([[-6.652204]], dtype=float32))
(65439, array([[-6.6491413]], dtype=float32))
(34107, array([[-6.6453114]], dtype=float32))
